In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
from pyspark.ml import PipelineModel
from pyspark.sql.functions import col

# 1. Iniciar Sesión de Spark
spark = SparkSession.builder \
    .appName("SECOP_Validacion_Transformacion") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

# 2. Cargar los datos que generamos en el Notebook 03
# Esta es la ruta donde guardamos el archivo final procesado
path_features = "/opt/spark-data/processed/secop_features.parquet"
df_transformed = spark.read.parquet(path_features)

print(f" Registros cargados para validación: {df_transformed.count():,}")
print(f" Columnas encontradas: {len(df_transformed.columns)}")

# 3. Mostrar la estructura de los datos (Esquema)
# Aquí verificamos que existan las columnas _idx, _vec y features_raw
df_transformed.printSchema()

# 4. RETO: Aplicar Escalado (Opcional pero recomendado para Regresión)
# El StandardScaler hace que todas las variables numéricas estén en la misma escala
scaler = StandardScaler(
    inputCol="features_raw", 
    outputCol="features_scaled", 
    withStd=True, 
    withMean=False
)

# Entrenamos el escalador con los datos cargados
scaler_model = scaler.fit(df_transformed)
df_final = scaler_model.transform(df_transformed)

# 5. Ver el resultado final antes de la Regresión
print("\n=== MUESTRA DE DATOS ESCALADOS Y LISTOS ===")
df_final.select("features_raw", "features_scaled", "valor_del_contrato").show(5, truncate=False)

# Guardar el dataset final (ya escalado) para el Notebook 05
output_final = "/opt/spark-data/processed/secop_final_ready.parquet"
df_final.write.mode("overwrite").parquet(output_final)

print(f" ¡Validación completa! Datos guardados en: {output_final}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/14 15:33:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


 Registros cargados para validación: 967
 Columnas encontradas: 16
root
 |-- referencia_del_contrato: string (nullable = true)
 |-- nit_entidad: string (nullable = true)
 |-- nombre_entidad: string (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciudad: string (nullable = true)
 |-- tipo_de_contrato: string (nullable = true)
 |-- valor_del_contrato: double (nullable = true)
 |-- fecha_de_firma: string (nullable = true)
 |-- estado_contrato: string (nullable = true)
 |-- departamento_idx: double (nullable = true)
 |-- tipo_de_contrato_idx: double (nullable = true)
 |-- estado_contrato_idx: double (nullable = true)
 |-- departamento_vec: vector (nullable = true)
 |-- tipo_de_contrato_vec: vector (nullable = true)
 |-- estado_contrato_vec: vector (nullable = true)
 |-- features_raw: vector (nullable = true)




=== MUESTRA DE DATOS ESCALADOS Y LISTOS ===


26/02/14 15:33:31 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+-------------------------------------------+-------------------------------------------------------------------------------------------------+------------------+
|features_raw                               |features_scaled                                                                                  |valor_del_contrato|
+-------------------------------------------+-------------------------------------------------------------------------------------------------+------------------+
|(53,[0,32,48,52],[1.0,1.0,1.0,7.3451015E7])|(53,[0,32,48,52],[2.115887659944155,2.939577643658159,11.790091178612657,0.0012021355132655794]) |7.3451015E7       |
|(53,[26,32,43,52],[1.0,1.0,1.0,6673341.0]) |(53,[26,32,43,52],[13.935715195287179,2.939577643658159,2.340529601977721,1.0921918789319976E-4])|6673341.0         |
|(53,[0,32,42,52],[1.0,1.0,1.0,1.32E7])     |(53,[0,32,42,52],[2.115887659944155,2.939577643658159,2.127481987864246,2.1603770587929447E-4])  |1.32E7            |
|(53,[0,32,42,52],[1.0

 ¡Validación completa! Datos guardados en: /opt/spark-data/processed/secop_final_ready.parquet
